# Домашнее задание по теме "Основы нейронных сетей"

In [371]:
#import kagglehub
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

## Загрузка данных

In [372]:
# path_boston = kagglehub.dataset_download("vikrishnan/boston-house-prices")
# print("Path to dataset files:", path_boston)

In [373]:
# ! mv /home/andrew/.cache/kagglehub/datasets/vikrishnan/boston-house-prices/versions/1/housing.csv datas/housing_boston.csv

## Подготовка данных

In [374]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df = pd.read_csv('datas/housing_boston.csv', delimiter=r"\s+", names=column_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [375]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


Пропущенных значений нет. Для загрузи данных в массив можно испльзовать `np.loadtxt` или `np.genfromtxt`

In [376]:
housing_numpy_01 = np.loadtxt('datas/housing_boston.csv', dtype=np.float32)
housing_numpy_01.shape

(506, 14)

In [377]:
housing_numpy_02 = np.genfromtxt('datas/housing_boston.csv', dtype=np.float32)
housing_numpy_02.shape

(506, 14)

### Создаем тензор

In [378]:
housing = torch.tensor(housing_numpy_01, dtype=torch.float32)
housing.shape

torch.Size([506, 14])

In [379]:
# Входные данные
inputs = housing[:, : -1]
inputs.shape

torch.Size([506, 13])

In [380]:
inputs_norm = torch.nn.functional.normalize(inputs)
inputs_norm.shape

torch.Size([506, 13])

In [381]:
# Выходные данные
labels = housing[:, -1]
labels.shape

torch.Size([506])

In [382]:
# Выровняем размерность
labels = labels.unsqueeze(1)
labels.shape

torch.Size([506, 1])

## Делим данные на train и val выборки

In [383]:
# Размер валидационной выборки
n_val = int(0.2 * inputs.shape[0])
n_val

101

In [384]:
# Перемешиваем индексы
shuffled_indeces = torch.randperm(inputs.shape[0])
# shuffled_indeces

In [385]:
train_indeces = shuffled_indeces[: -n_val]
val_indeces = shuffled_indeces[-n_val :]

In [386]:
train_inputs = inputs_norm[train_indeces]
train_labels = labels[train_indeces]

In [387]:
val_inputs = inputs_norm[val_indeces]
val_labels = labels[val_indeces]

In [388]:
# ds = TensorDataset(input, label)

In [389]:
# train_ds, val_ds = random_split(ds, [int(input.shape[0] * 0.8), input.shape[0] - (int(input.shape[0] * 0.8))])

In [390]:
# int(input.shape[0]) - int(input.shape[0] * 0.8)

In [391]:
train_ds = TensorDataset(train_inputs, train_labels)
val_ds = TensorDataset(val_inputs, val_labels)

In [ ]:
batch_size = 100
train_iter = DataLoader(train_ds, batch_size, shuffle=True)

## Создание модели

In [393]:
model = torch.nn.Sequential(torch.nn.Linear(13, 1))
model

Sequential(
  (0): Linear(in_features=13, out_features=1, bias=True)
)

In [394]:
loss = torch.nn.MSELoss(reduction='mean')

In [397]:
trainer = torch.optim.Adam(model.parameters(), lr=0.001)

## Обучение модели

In [ ]:
num_epoch = 10000
for epoch in range(1, num_epoch +1):
    for X, y in train_iter:
        trainer.zero_grad()
        l = loss(model(X).reshape(-1), y)
        l.backward()
        trainer.step()
    l = loss(model(train_inputs).reshape(-1), train_labels)
    if epoch % 5 == 0:
        print('Epoch %d, Loss: %f' % (epoch, l.item()),
              '|\tw', model[0].weight.data,
              '|\tb', model[0].bias.data)

Epoch 5, Loss: 355.522736 |	w tensor([[2.3472, 2.0636, 2.6793, 2.0171, 2.5552, 2.2650, 2.4207, 2.3428, 2.4571,
         2.7442, 2.7295, 2.4931, 2.2315]]) |	b tensor([2.2275])
Epoch 10, Loss: 353.407837 |	w tensor([[2.3709, 2.0880, 2.7045, 2.0378, 2.5802, 2.2900, 2.4456, 2.3678, 2.4808,
         2.7688, 2.7544, 2.5182, 2.2565]]) |	b tensor([2.2524])
Epoch 15, Loss: 351.320496 |	w tensor([[2.3959, 2.1094, 2.7292, 2.0564, 2.6050, 2.3144, 2.4703, 2.3920, 2.5051,
         2.7935, 2.7790, 2.5427, 2.2811]]) |	b tensor([2.2770])
Epoch 20, Loss: 349.254761 |	w tensor([[2.4187, 2.1311, 2.7538, 2.0740, 2.6294, 2.3387, 2.4949, 2.4158, 2.5292,
         2.8179, 2.8035, 2.5671, 2.3058]]) |	b tensor([2.3015])
Epoch 25, Loss: 347.217499 |	w tensor([[2.4427, 2.1521, 2.7786, 2.0932, 2.6538, 2.3628, 2.5194, 2.4396, 2.5535,
         2.8422, 2.8280, 2.5911, 2.3310]]) |	b tensor([2.3257])
Epoch 30, Loss: 345.135681 |	w tensor([[2.4676, 2.1751, 2.8034, 2.1128, 2.6786, 2.3876, 2.5443, 2.4643, 2.5789,
         

## Проверка качества модели

In [410]:
val_iter = DataLoader(val_ds, batch_size=val_inputs.shape[0])
for X, y in val_iter:
        l = loss(model(X).reshape(-1), y)
        print(f"Validation loss {l}")
        

Validation loss 75.13533782958984
